In [1]:
import collections

# import helper
import numpy as np
# import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [2]:
import pandas as pd

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4870404041812672980
]


In [4]:
def load_data(path):
    with open(path, 'r') as fd:
      return fd.readlines()

In [5]:
df=pd.read_csv('processed_title.csv')

In [6]:
df1=df

In [7]:
english_sentences=df1['Original']
french_sentences=df1['Target']

In [8]:
english_sentences

0                 Delete all elements at application close
1        Which one of thesecode samples has better perf...
2        Why library name gets an additional UNK NUMBER...
3                What have you use Regular Expressions for
4        Calculation of cubic bezier with known halfway...
                               ...                        
22610    How to refresh a webpage automaticly every spe...
22611    Help me to complete my UITableView to Navigati...
22612    JQuery focus() on jQuery-applied anchors works...
22613                 (tcl/Expect) clear screen after exit
22614    How to call C# method taking an object of type...
Name: Original, Length: 22615, dtype: object

In [9]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

210794 English words.
26489 unique English words.
10 Most common words in the English dataset:
"to" "a" "in" "How" "the" "of" "and" "for" "NUMBER" "with"

216743 French words.
21989 unique French words.
10 Most common words in the French dataset:
"to" "a" "in" "How" "the" "of" "I" "and" "NUMBER" "for"


In [10]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer(num_words=None, char_level=True,lower=False)
#     tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    print(tokenizer.texts_to_sequences(x))
    return tokenizer.texts_to_sequences(x), tokenizer




In [11]:
all_column_to_list=df1['Original'].tolist()+df1['Target'].tolist()

In [12]:
preprocess_x, x_tk = tokenize(all_column_to_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
x_tk.index_word

{1: ' ',
 2: 'e',
 3: 't',
 4: 'o',
 5: 'a',
 6: 'i',
 7: 'n',
 8: 'r',
 9: 's',
 10: 'l',
 11: 'c',
 12: 'd',
 13: 'h',
 14: 'u',
 15: 'p',
 16: 'm',
 17: 'g',
 18: 'w',
 19: 'f',
 20: 'y',
 21: 'b',
 22: 'v',
 23: 'S',
 24: 'N',
 25: 'U',
 26: 'I',
 27: 'M',
 28: 'H',
 29: 'C',
 30: 'E',
 31: 'R',
 32: 'k',
 33: 'P',
 34: 'B',
 35: 'x',
 36: 'L',
 37: 'W',
 38: 'A',
 39: 'D',
 40: 'K',
 41: 'T',
 42: '-',
 43: 'j',
 44: 'Q',
 45: 'F',
 46: 'q',
 47: "'",
 48: 'O',
 49: 'J',
 50: '(',
 51: ':',
 52: ',',
 53: 'V',
 54: ')',
 55: '+',
 56: '"',
 57: '/',
 58: 'G',
 59: '#',
 60: 'z',
 61: 'X',
 62: '_',
 63: ']',
 64: '[',
 65: '>',
 66: '<',
 67: 'Y',
 68: '*',
 69: '&',
 70: 'Z',
 71: '$',
 72: '`',
 73: '@',
 74: '%',
 75: '\\',
 76: '|',
 77: '~',
 78: '^',
 79: '\x7f'}

In [14]:
def tokenize_toseq(x,Tok):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    
#     tokenizer = Tokenizer()
    Tok.fit_on_texts(x)
#     print(Tok.texts_to_sequences(x))
    return Tok.texts_to_sequences(x)




In [15]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding='post')

# test_pad(pad)

# # Pad Tokenized output
# test_pad = pad(text_tokenized)
# for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
#     print('Sequence {} in x'.format(sample_i + 1))
#     print('  Input:  {}'.format(np.array(token_sent)))
#     print('  Output: {}'.format(pad_sent))

In [18]:
preproc_english_sentences1, preproc_french_sentences1, english_tokenizer1, french_tokenizer =\
    preprocess1(english_sentences, french_sentences,x_tk)

In [17]:
def preprocess1(x,y,x_tk):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x = tokenize_toseq(x,x_tk)
    preprocess_y = tokenize_toseq(y,x_tk)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    return preprocess_x, preprocess_y, x_tk, x_tk

In [20]:
english_tokenizer1.index_word

{1: ' ',
 2: 'e',
 3: 't',
 4: 'o',
 5: 'a',
 6: 'i',
 7: 'n',
 8: 'r',
 9: 's',
 10: 'l',
 11: 'c',
 12: 'd',
 13: 'h',
 14: 'u',
 15: 'p',
 16: 'm',
 17: 'g',
 18: 'w',
 19: 'f',
 20: 'y',
 21: 'b',
 22: 'v',
 23: 'S',
 24: 'N',
 25: 'U',
 26: 'I',
 27: 'M',
 28: 'H',
 29: 'C',
 30: 'E',
 31: 'R',
 32: 'k',
 33: 'P',
 34: 'B',
 35: 'x',
 36: 'L',
 37: 'W',
 38: 'A',
 39: 'D',
 40: 'K',
 41: 'T',
 42: '-',
 43: 'j',
 44: 'Q',
 45: 'F',
 46: 'q',
 47: "'",
 48: 'O',
 49: 'J',
 50: '(',
 51: ':',
 52: ',',
 53: 'V',
 54: ')',
 55: '+',
 56: '"',
 57: '/',
 58: 'G',
 59: '#',
 60: 'z',
 61: 'X',
 62: '_',
 63: ']',
 64: '[',
 65: '>',
 66: '<',
 67: 'Y',
 68: '*',
 69: '&',
 70: 'Z',
 71: '$',
 72: '`',
 73: '@',
 74: '%',
 75: '\\',
 76: '|',
 77: '~',
 78: '^',
 79: '\x7f'}

In [19]:
french_tokenizer.index_word

{1: ' ',
 2: 'e',
 3: 't',
 4: 'o',
 5: 'a',
 6: 'i',
 7: 'n',
 8: 'r',
 9: 's',
 10: 'l',
 11: 'c',
 12: 'd',
 13: 'h',
 14: 'u',
 15: 'p',
 16: 'm',
 17: 'g',
 18: 'w',
 19: 'f',
 20: 'y',
 21: 'b',
 22: 'v',
 23: 'S',
 24: 'N',
 25: 'U',
 26: 'I',
 27: 'M',
 28: 'H',
 29: 'C',
 30: 'E',
 31: 'R',
 32: 'k',
 33: 'P',
 34: 'B',
 35: 'x',
 36: 'L',
 37: 'W',
 38: 'A',
 39: 'D',
 40: 'K',
 41: 'T',
 42: '-',
 43: 'j',
 44: 'Q',
 45: 'F',
 46: 'q',
 47: "'",
 48: 'O',
 49: 'J',
 50: '(',
 51: ':',
 52: ',',
 53: 'V',
 54: ')',
 55: '+',
 56: '"',
 57: '/',
 58: 'G',
 59: '#',
 60: 'z',
 61: 'X',
 62: '_',
 63: ']',
 64: '[',
 65: '>',
 66: '<',
 67: 'Y',
 68: '*',
 69: '&',
 70: 'Z',
 71: '$',
 72: '`',
 73: '@',
 74: '%',
 75: '\\',
 76: '|',
 77: '~',
 78: '^',
 79: '\x7f'}

In [20]:
max_english_sequence_length = preproc_english_sentences1.shape[1]
max_french_sequence_length = preproc_french_sentences1.shape[1]
english_vocab_size = len(english_tokenizer1.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 183
Max French sentence length: 180
English vocabulary size: 79
French vocabulary size: 79


In [21]:
import pandas as pd
import collections
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense,  RepeatVector, TimeDistributed
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


In [22]:
tmp_x1 = tf.keras.utils.to_categorical(preproc_english_sentences1[0])

In [24]:
tmp_x = tf.keras.utils.to_categorical(preproc_english_sentences1)
tmp_y = tf.keras.utils.to_categorical(preproc_french_sentences1)

In [25]:
def encdec_model(input_shape, output_sequence_length, original_vocab_size, target_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param original_vocab_size: Number of unique character in original text
    :param target_vocab_size: Number of unique character in target text
    :return: Keras model built, but not trained
    """    
    # Hyperparameters
    learning_rate = 0.001
    
    model = Sequential()
    model.add(LSTM(256, input_shape=input_shape))
    model.add(RepeatVector(output_sequence_length))
    model.add(SimpleRNN(1024, return_sequences=True))
    model.add(Dense(1024, activation='relu'))
    model.add(LSTM(256, return_sequences=True))
    model.add(TimeDistributed(Dense(target_vocab_size, activation='softmax')))

    model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate), metrics=['accuracy'])
    return model

    
    return model

In [26]:
from keras.models import Sequential
from keras.layers import Activation, SimpleRNN

# Train and Print prediction(s)
encdec_rnn_model = encdec_model(
    tmp_x.shape[1:],
    max_french_sequence_length,
    english_vocab_size + 1,
    french_vocab_size )

encdec_rnn_model.summary()

encdec_rnn_model.fit(tmp_x, tmp_y, batch_size=32, epochs=2, validation_split=0.2)

encdec_rnn_model.save('Model_char_lvl_Org_Mode.h5') 

In [27]:
from keras.models import load_model


# returns a compiled model
# identical to the previous one
encdec_rnn_model = load_model('Model_char_lvl_Org_Mode.h5')


In [28]:
x_tk.to_json

<bound method Tokenizer.to_json of <keras_preprocessing.text.Tokenizer object at 0x000001AFD97417F0>>

In [29]:
import io
import json

In [40]:
tokenizer_json = x_tk.to_json()
with io.open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

In [29]:
def tokenizer_from_json(json_string):
  """Parses a JSON tokenizer configuration and returns a tokenizer instance.
  Deprecated: `tf.keras.preprocessing.text.Tokenizer` does not operate on
  tensors and is not recommended for new code. Prefer
  `tf.keras.layers.TextVectorization` which provides equivalent functionality
  through a layer which accepts `tf.Tensor` input. See the
  [text loading tutorial](https://www.tensorflow.org/tutorials/load_data/text)
  for an overview of the layer and text handling in tensorflow.
  Args:
      json_string: JSON string encoding a tokenizer configuration.
  Returns:
      A Keras Tokenizer instance
  """
  tokenizer_config = json.loads(json_string)
  config = tokenizer_config.get('config')

  word_counts = json.loads(config.pop('word_counts'))
  word_docs = json.loads(config.pop('word_docs'))
  index_docs = json.loads(config.pop('index_docs'))
  # Integer indexing gets converted to strings with json.dumps()
  index_docs = {int(k): v for k, v in index_docs.items()}
  index_word = json.loads(config.pop('index_word'))
  index_word = {int(k): v for k, v in index_word.items()}
  word_index = json.loads(config.pop('word_index'))

  tokenizer = Tokenizer(**config)
  tokenizer.word_counts = word_counts
  tokenizer.word_docs = word_docs
  tokenizer.index_docs = index_docs
  tokenizer.word_index = word_index
  tokenizer.index_word = index_word
  return tokenizer

In [38]:
with open('tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

In [30]:
import collections

# import helper
import numpy as np
# import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import pandas as pd
import collections
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense,  RepeatVector, TimeDistributed
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


In [31]:
import pandas as pd

In [32]:
preprocess_x1 = tokenize_toseq(io,x_tk)

TypeError: 'module' object is not iterable

In [33]:
preprocess_x1

NameError: name 'preprocess_x1' is not defined

In [49]:
preprocess_x1 = pad(preprocess_x1)

In [50]:
preprocess_x1

array([[37],
       [13],
       [ 6],
       [11],
       [13],
       [ 1],
       [ 4],
       [ 7],
       [ 2],
       [ 1],
       [ 4],
       [19],
       [ 1],
       [ 3],
       [13],
       [ 2],
       [ 9],
       [ 2],
       [11],
       [ 4],
       [12],
       [ 2],
       [ 1],
       [ 9],
       [ 5],
       [16],
       [15],
       [10],
       [ 2],
       [ 9],
       [ 1],
       [13],
       [ 5],
       [ 9],
       [ 1],
       [21],
       [ 2],
       [ 3],
       [ 3],
       [ 2],
       [ 8],
       [ 1],
       [15],
       [ 2],
       [ 8],
       [19],
       [ 4],
       [ 8],
       [16],
       [ 5],
       [ 7],
       [11],
       [ 2]])

In [51]:
tmp_x1 = tf.keras.utils.to_categorical(preprocess_x1)

In [52]:
tmp_x1

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [78]:
def sentence_pred(data):
    
#     print(data)
    sentence=[tokenizer.word_index[ch] for ch in data]
    oo=sentence
#     print("???")
# #     print(oo)
#     print("???")
#     sentence = [english_tokenizer.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=preproc_english_sentences1.shape[-1], padding='post')
    print("////////")

    print(sentence)
    
    print("///////")
    sentences = np.array([sentence[0], preproc_english_sentences1[0]])
    print("???")

    print(sentences.reshape(1,2,3))
   
    print("???")
    predictions = encdec_rnn_model.predict(sentences, len(sentences))
    RX=""
    for prediction in np.argmax(predictions[0], 1):
        print(prediction)
        dc=logits_to_text(prediction,x_tk)
        dc=str(dc)
#         print(dc)
        if dc!=None:
#             RX+=dc+" "
            RX+=dc
#     print(RX) 
    sentence = " ".join(RX.split())
    return sentence


xi=sentence_pred(io)

////////
[[37 13  6 11 13  1  4  7  2  1  4 19  1  3 13  2  9  2 11  4 12  2  1  9
   5 16 15 10  2  9  1 13  5  9  1 21  2  3  3  2  8  1 15  2  8 19  4  8
  16  5  7 11  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
///////
???


ValueError: cannot reshape array of size 366 into shape (1,2,3)

In [40]:
def logits_to_text(logits,tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
#     print(tokenizer.index_word.get(logits))
    
    r=tokenizer.index_word.get(logits)

    if r!=None:
        r=str(r)
    else:
        r=""
    return r

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [35]:
io=df['Original'][1]

In [36]:
io

'Which one of thesecode samples has better performance'

In [39]:
xi=sentence_pred(io)

ValueError: in user code:

    C:\Users\subra\Anaconda\lib\site-packages\keras\engine\training.py:1586 predict_function  *
        return step_function(self, iterator)
    C:\Users\subra\Anaconda\lib\site-packages\keras\engine\training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\subra\Anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\subra\Anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\subra\Anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\subra\Anaconda\lib\site-packages\keras\engine\training.py:1569 run_step  **
        outputs = model.predict_step(data)
    C:\Users\

In [47]:
tmp_x[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [54]:
tmp_x1

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [66]:
encdec_rnn_model.predict(tmp_x[500:550])

array([[[8.68646530e-05, 8.45148170e-04, 2.48419214e-03, ...,
         1.13727707e-04, 8.14071609e-05, 1.02118742e-04],
        [1.69759107e-04, 6.60383003e-03, 9.06272680e-02, ...,
         6.68481562e-06, 8.84967722e-06, 6.52177778e-06],
        [3.92694463e-04, 7.81763494e-02, 6.41846359e-02, ...,
         1.39410731e-05, 1.04691599e-05, 6.84323822e-06],
        ...,
        [9.97937679e-01, 2.75836239e-04, 1.70196145e-04, ...,
         1.57941983e-07, 4.25538786e-08, 5.42085310e-08],
        [9.97940242e-01, 2.75462866e-04, 1.69926381e-04, ...,
         1.57768042e-07, 4.25026379e-08, 5.41419176e-08],
        [9.97942984e-01, 2.75116268e-04, 1.69675361e-04, ...,
         1.57607730e-07, 4.24554401e-08, 5.40797309e-08]],

       [[8.68646603e-05, 8.45148228e-04, 2.48419237e-03, ...,
         1.13727714e-04, 8.14071682e-05, 1.02118844e-04],
        [1.69759165e-04, 6.60383003e-03, 9.06272084e-02, ...,
         6.68481061e-06, 8.84967085e-06, 6.52177323e-06],
        [3.92694463e-04, 

In [67]:
pred

array([[[8.68646530e-05, 8.45148170e-04, 2.48419214e-03, ...,
         1.13727707e-04, 8.14071609e-05, 1.02118742e-04],
        [1.69759078e-04, 6.60382910e-03, 9.06272456e-02, ...,
         6.68480743e-06, 8.84967540e-06, 6.52177641e-06],
        [3.92694463e-04, 7.81763941e-02, 6.41845986e-02, ...,
         1.39410604e-05, 1.04691599e-05, 6.84323822e-06],
        ...,
        [9.97937679e-01, 2.75835977e-04, 1.70195985e-04, ...,
         1.57941983e-07, 4.25537969e-08, 5.42084280e-08],
        [9.97940242e-01, 2.75462866e-04, 1.69926381e-04, ...,
         1.57767914e-07, 4.25026379e-08, 5.41419176e-08],
        [9.97942984e-01, 2.75116530e-04, 1.69675521e-04, ...,
         1.57607872e-07, 4.24554401e-08, 5.40798339e-08]],

       [[8.68646675e-05, 8.45148345e-04, 2.48419377e-03, ...,
         1.13727729e-04, 8.14070954e-05, 1.02118858e-04],
        [1.69759078e-04, 6.60382910e-03, 9.06272903e-02, ...,
         6.68480743e-06, 8.84967540e-06, 6.52177641e-06],
        [3.92694754e-04, 

In [73]:
RX=""
for prediction in np.argmax(pred[40], 1):
        print(prediction)
        dc=logits_to_text(prediction,x_tk)
        dc=str(dc)
#         print(dc)
        if dc!=None:
#             RX+=dc+" "
            RX+=dc
print(RX) 
sentence = " ".join(RX.split())


28
4
18
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
How                              
